In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import glob

In [ ]:
'''
This cell  imports a list of high-speed streams observed by OMNI and compares them to a list of times when ARTEMIS is on the E-S line
'''

omni_hss = pd.read_csv('lists/filtered_hss_from_omni.txt', delim_whitespace=True, names=['year', 'dayofyear', 'hour', 'minute', 'Bz', 'RMS', 'Speed', 'Density', 'Temperature'])    # Import the list of HSS observed by OMNI
artemis_dates = pd.read_csv('../artemis-near-ESline.csv', header=0,  parse_dates=[0, 1]) # Import times when ARTEMIS was near the E-S line

omni_hss['datetime'] = pd.to_datetime(omni_hss['year'].astype(str)+'-'+omni_hss['dayofyear'].astype(str)+' '+omni_hss['hour'].astype(str).str.zfill(2)+':'+omni_hss['minute'].astype(str).str.zfill(2), format='%Y-%j %H:%M')   # Convert time columns to datetime objects
omni_hss.drop(columns=['year', 'dayofyear', 'hour', 'minute'], inplace=True)    # Drop unneeded columns now that we have datetime column

start_times = []    # Create empty array to store starting times for the HSS
stop_times = [] # Create empty array to store stopping times for the HSS
threshold = pd.Timedelta(minutes=180)   # HSS must be more than 180 minutes apart for them to be considered separate events
current_start = omni_hss['datetime'].iloc[0]    # Initialize the start of the first window for identifying overlapping ranges
prev_gap = pd.Timedelta(0)  # Initialize the time gap

# This loop goes through the list of HSSs and breaks them up into "events" which need to be at least 180 min apart
for i in range(1, len(omni_hss)):
    current_gap = omni_hss['datetime'].iloc[i] - omni_hss['datetime'].iloc[i - 1]   # Find the gap between the last timestamp and the current timestamp
    if current_gap > threshold and current_gap > prev_gap * 2:  # If this gap exceeds the threshold and is at least twice the size of the last gap (which should be on the order of minutes)
        start_times.append(current_start)   # Then create a new start time and append it to the array
        stop_times.append(omni_hss['datetime'].iloc[i - 1]) # Also take the last timestamp and add it as an end time to that array
        current_start = omni_hss['datetime'].iloc[i]  # Start a new group
    prev_gap = current_gap  # Reset the gap

start_times.append(current_start)   # Finally after the last iteration append the last value
stop_times.append(omni_hss['datetime'].iloc[-1])    # And do the same for the stop times.

# Create a DataFrame with start_times and stop_times
result = pd.DataFrame({'start_times': start_times, 'stop_times': stop_times})   # Create a new DataFrame based on these start and stop times from the loop

valid_starts = []   # Empty storage array to house the starts and stops in the DataFrame that are valid and overlap the ARTEMIS data
valid_stops = []    # Same as the valid_starts array but for stops

# This loop iterates through this list of starts and stops when ARTEMIS is on the E-S line and compares them to the intervals of HSSs
for astart, astop in zip(artemis_dates['Date_Start'], artemis_dates['Date_Stop']):
    for hstart, hstop in zip(result['start_times'], result['stop_times']):
        if (astart <= hstop) and (astop >= hstart): # Check if intervals overlap
            valid_start = max(astart, hstart)   # Find the latter of the two start times
            valid_stop = min(astop, hstop)  # Find the earlier of the two end times

            valid_starts.append(dt.datetime.strftime(valid_start, format='%Y-%m-%d %H:%M:%S'))
            valid_stops.append(dt.datetime.strftime(valid_stop, format='%Y-%m-%d %H:%M:%S'))

            fig, ax = plt.subplots(figsize=(15, 2))
            ax.barh(0.7, (hstop - hstart), left=hstart, height=0.4, color='red', alpha=0.6, label='HSS Candidate')
            ax.barh(0.3, (astop - astart), left=astart, height=0.4, color='blue', alpha=0.6, label='ARTEMIS')

            ax.set_title(f'HSS on {dt.datetime.strftime(valid_start, format="%Y-%m-%d")}')
            ax.set_yticks([])  # Remove y-axis ticks since they're not meaningful
            ax.set_xlabel('Date and Time (UTC)')
            ax.grid(True, alpha=0.3)
            ax.legend(loc='upper left')

            # Format x-axis to show dates nicely
            #date_formatter = mdates.DateFormatter('%Y-%m-%d %H:%M')
            #ax.xaxis.set_major_formatter(date_formatter)

            ax.xaxis.set_major_locator(mdates.HourLocator(interval=6))
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H:%M'))
            ax.xaxis.set_minor_locator(mdates.HourLocator(interval=1))
            #plt.gcf().autofmt_xdate()

            # Adjust layout to prevent date labels from being cut off
            plt.tight_layout()

            plt.savefig(f'lists/overlap_plots/event_{dt.datetime.strftime(valid_start, format="%Y-%m-%d")}.png', bbox_inches='tight', dpi=300)
            plt.close()

intervals = pd.DataFrame({'Artemis_HSS_start': valid_starts, 'Artemis_HSS_stop': valid_stops})  # Create a DataFrame with all the valid starts and stops
intervals.to_csv('lists/HSS_intervals_from_filtered_omni.csv', index=None) # Convert the dataframe to a CSV

In [8]:
'''
This cell looks through all the existing OMNI data in ../Ordinary Solar Wind/ and looks for a HSS. The process works like this:
-> Is Vx >= 450 km/s
    |-> If so, check the density is less than 3 particles/cm^3
        | -> If so, check the next point and repeat. When you reach the end, if the interval length is less than one hour don't save it, but if it is greater, save it.
'''
def check_hss_conditions(row):  #Check if a row meets HSS conditions
    return (abs(row['VX']) >= 450 and row['N'] <= 5)

# Define directories
omni_input_dir = "../Ordinary Solar Wind/omni"
artemis_input_dir = "../Ordinary Solar Wind/artemis"
base_output_dir = "candidate_solar_wind/from_ordinary_omni"
omni_output_dir = os.path.join(base_output_dir, "omni")
artemis_output_dir = os.path.join(base_output_dir, "artemis")

# Create output directories
os.makedirs(omni_output_dir, exist_ok=True)
os.makedirs(artemis_output_dir, exist_ok=True)

# Get all CSV files
omni_files = glob.glob(os.path.join(omni_input_dir, "omni_*.csv"))
artemis_files = glob.glob(os.path.join(artemis_input_dir, "artemis_*.csv"))

start_times = []
stop_times = []

# Process OMNI files first to get intervals
for file in omni_files:
    df = pd.read_csv(file)
    print(f"Opening OMNI file {file}")
    df['Time'] = pd.to_datetime(df['Time'])
    df['is_hss'] = df.apply(check_hss_conditions, axis=1)
    df['group'] = (df['is_hss'] != df['is_hss'].shift()).cumsum()

    for group_num in df[df['is_hss']]['group'].unique():
        hss_period = df[df['group'] == group_num]

        if not hss_period['is_hss'].iloc[0]:
            continue

        duration = (hss_period['Time'].max() - hss_period['Time'].min())

        if duration >= dt.timedelta(hours=1):
            start_time = hss_period['Time'].iloc[0]
            stop_time = hss_period['Time'].iloc[-1]

            start_times.append(start_time)
            stop_times.append(stop_time)

            # Save OMNI data
            output_filename = f"omni_{start_time.strftime('%Y-%m-%d_%H-%M')}.csv"
            output_path = os.path.join(omni_output_dir, output_filename)
            hss_period.drop(['is_hss', 'group'], axis=1).to_csv(output_path, index=False)

            print(f"Found HSS period: {start_time} to {stop_time}")
        else:
            print(f"Found no HSS periods...")

# Create intervals DataFrame
intervals = pd.DataFrame({
    'HSS_start': [dt.datetime.strftime(t, '%Y-%m-%d %H:%M:%S') for t in start_times],
    'HSS_stop': [dt.datetime.strftime(t, '%Y-%m-%d %H:%M:%S') for t in stop_times]
})
intervals.to_csv('lists/HSS_intervals_from_ordinary_omni.csv', index=False)

# Process ARTEMIS files and extract matching intervals
for file in artemis_files:
    df_artemis = pd.read_csv(file)
    print(f"Opening ARTEMIS file {file}")
    df_artemis['Time'] = pd.to_datetime(df_artemis['Time'])

    # For each HSS interval, extract matching ARTEMIS data
    for start, stop in zip(start_times, stop_times):
        mask = (df_artemis['Time'] >= start) & (df_artemis['Time'] <= stop)
        if mask.any():
            artemis_period = df_artemis[mask]

            # Save ARTEMIS data for this period
            output_filename = f"artemis_{start.strftime('%Y-%m-%d_%H-%M')}.csv"
            output_path = os.path.join(artemis_output_dir, output_filename)
            artemis_period.to_csv(output_path, index=False)

Opening OMNI file ../Ordinary Solar Wind/omni/omni_2012-12-12_00-00.csv
Opening OMNI file ../Ordinary Solar Wind/omni/omni_2016-12-28_00-00.csv
Found HSS period: 2016-12-28 00:38:00 to 2016-12-28 11:30:00
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Opening OMNI file ../Ordinary Solar Wind/omni/omni_2013-06-07_00-00.csv
Found no HSS periods...
Opening OMNI file ../Ordinary Solar Wind/omni/omni_2020-09-16_00-00.csv
Opening OMNI file ../Ordinary Solar Wind/omni/omni_2019-07-31_00-00.csv
Found no HSS periods...
Found no HSS periods...
Found no HSS periods...
Fou

In [2]:
'''
This cell calculates dB/B in a running one-hour window with centered windows
'''
variables = ['BY_GSM', 'BZ_GSM', 'VX', 'N']
labels = ['B_y', 'B_z', 'V_x', 'N']

input_dir = "candidate_solar_wind/final/omni/"
files = glob.glob(os.path.join(input_dir, "omni_*.csv"))

window_size = 60  # 60-minute window
half_window = window_size // 2  # 30 minutes on each side

for f in files:
    print(f"Processing {f}")
    omni = pd.read_csv(f, delimiter=',', header=0, parse_dates=True)
    n_windows = len(omni) - window_size + 1  # Total possible windows

    if n_windows >= 1:
        print(f'Computing {n_windows} windows...')
        for v, l in zip(variables, labels):
            data_rows = [np.nan] * half_window  # Pad start with NaN

            # Calculate centered windows
            for n in range(n_windows):
                o_slice = omni[v][n:n + window_size]  # Take a chunk of data 60 minutes long
                avg_o = np.mean(o_slice)
                deltaO = np.array([x - avg_o for x in o_slice])  # Convert to numpy array
                deltaO = np.array([x - avg_o for x in o_slice])  # Convert to numpy array
                deltaO_over_O = np.mean(np.abs(deltaO)) / np.abs(avg_o)
                data_rows.append(deltaO_over_O)

            # Pad end with NaN
            data_rows.extend([np.nan] * (half_window - 1))

            # Ensure the length matches the original dataframe
            assert len(data_rows) == len(omni)

            # Add the new column to the dataframe
            omni[f'delta{l}'] = data_rows

        # Save the updated dataframe back to the same file
        omni.to_csv(f, index=False)
        print(f"Saved updated file with new columns: {f}")


Processing candidate_solar_wind/final/omni/omni_2015-07-16_13-00.csv
Computing 811 windows...
Saved updated file with new columns: candidate_solar_wind/final/omni/omni_2015-07-16_13-00.csv
Processing candidate_solar_wind/final/omni/omni_2014-01-02_21-00.csv
Computing 1350 windows...
Saved updated file with new columns: candidate_solar_wind/final/omni/omni_2014-01-02_21-00.csv
Processing candidate_solar_wind/final/omni/omni_2017-02-26_00-00.csv
Computing 330 windows...
Saved updated file with new columns: candidate_solar_wind/final/omni/omni_2017-02-26_00-00.csv
Processing candidate_solar_wind/final/omni/omni_2015-10-14_03-00.csv
Computing 511 windows...
Saved updated file with new columns: candidate_solar_wind/final/omni/omni_2015-10-14_03-00.csv
Processing candidate_solar_wind/final/omni/omni_2015-11-12_01-00.csv
Computing 1105 windows...
Saved updated file with new columns: candidate_solar_wind/final/omni/omni_2015-11-12_01-00.csv
Processing candidate_solar_wind/final/omni/omni_2022-

In [3]:
input_dir = "candidate_solar_wind/final/omni/"
files = glob.glob(os.path.join(input_dir, "omni_*.csv"))

for f in files:
    omni = pd.read_csv(f, delimiter=',', header=0)
    omni = omni.drop(columns=['Unnamed: 0']).reset_index(drop=True)
    omni.to_csv(f, index=True)